In [1]:
import src.functions_search as fs
import src.functions_rank as fr
import src.functions_mongo as fm
import googlemaps
import geopy.distance
from pymongo import MongoClient
from datetime import datetime
import os
import requests
import json
import time
import pandas as pd
import folium
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
airports=pd.read_csv("./output/filtered_airports.csv")
offices=pd.read_csv("./output/filtered_offices.csv")

In [3]:
print(len(offices),len(airports))
display(offices)

85 50


,Unnamed: 0,name,category,city,country,latitude,longitude,is_gaming,total_companies
0,0,Thoof,web,Austin,USA,30.268735,-97.745209,0,36
1,1,RockYou,games_video,Redwood City,USA,37.484619,-122.206893,1,23
2,2,Carbonite,network_hosting,Boston,USA,42.350571,-71.076869,0,26
3,3,oDesk,web,Redwood City,USA,37.479952,-122.178557,0,23
4,4,Bazaarvoice,software,Austin,USA,30.407545,-97.717667,0,36
...,...,...,...,...,...,...,...,...,...
80,80,Virdia,cleantech,Redwood City,USA,37.523311,-122.259154,0,23
81,81,Buzzient,enterprise,Boston,USA,42.362856,-71.083762,0,26
82,82,Nuventix,cleantech,Austin,USA,30.239229,-97.834211,0,36
83,83,Pearl Therapeutics,biotech,Redwood City,USA,37.502169,-122.217727,0,23


In [17]:
print(offices.iloc[0])
print(airports.iloc[0])
print(fs.closest_airport(offices.iloc[0],airports))

Unnamed: 0                   17
name                        Qik
category                 mobile
city               Redwood City
country                     USA
latitude                37.4864
longitude               -122.23
is_gaming                     0
total_companies              23
dist_airport            20.1946
dist_starbucks          15.7279
dist_vegan              156.383
dist_school             635.726
rank                         22
Name: 17, dtype: object
Unnamed: 0                                        0
code                                            ABQ
lat                                         35.0494
lon                                        -106.625
name              Albuquerque International Airport
city                                    Albuquerque
state                                    New Mexico
country                               United States
woeid                                      12518564
icao                                           KABQ
direct_f

In [18]:
dist_airport=[]
for x in range(len(offices)):
    office=offices.iloc[x]
    dist_airport.append(fs.closest_airport(office,airports))
print(len(dist_airport))

85


In [19]:
offices['dist_airport'] = dist_airport 

In [20]:
display(offices.head())

,Unnamed: 0,name,category,city,country,latitude,longitude,is_gaming,total_companies,dist_airport,dist_starbucks,dist_vegan,dist_school,rank
17,17,Qik,mobile,Redwood City,USA,37.486368,-122.230116,0,23,20.194579,15.727920,156.383293,635.725913,22
68,68,SCVNGR,games_video,Boston,USA,42.340653,-71.068003,1,26,4.764528,593.158218,40.895928,1892.669333,22
73,73,Shareaholic,web,Boston,USA,42.364756,-71.103259,0,26,6.238827,1225.057454,80.897534,2312.798032,20
35,35,Optaros,consulting,Boston,USA,42.356812,-71.058185,0,26,2.979413,147.519598,174.009406,2820.901034,20
2,2,Carbonite,network_hosting,Boston,USA,42.350571,-71.076869,0,26,4.652931,360.800203,177.114680,1132.796641,19


In [21]:
##Function to find closest airport                 
def closest_starbucks(office):
    coords_off = (office.get('latitude'),office.get('longitude'))
    coords_off2 = ("{},{}".format(coords_off[0],coords_off[1]))
    nbs_list=fs.nearbysearchName("starbucks",coords_off2,1000)
    dist = []
    a=0
    if nbs_list==[]:
        dist.append(a)
    else:
        starbucks=pd.DataFrame(fs.getLocation(nbs_list)).iloc[0]
        coords_str=(starbucks.get('latitude'),starbucks.get('longitude'))
        geo_dist=(geopy.distance.geodesic(coords_off, coords_str).m)
        dist.append(geo_dist)
        return dist[0]


In [9]:
dist_starbucks=[]
for x in range(len(offices)):
    office=offices.iloc[x]
    dist_starbucks.append(fs.closest_starbucks(office))

offices['dist_starbucks'] = dist_starbucks

display(offices)

,Unnamed: 0,name,category,city,country,latitude,longitude,is_gaming,total_companies,dist_airport,dist_starbucks
0,0,Thoof,web,Austin,USA,30.268735,-97.745209,0,36,10.618504,223.125453
1,1,RockYou,games_video,Redwood City,USA,37.484619,-122.206893,1,23,21.824782,1392.763284
2,2,Carbonite,network_hosting,Boston,USA,42.350571,-71.076869,0,26,4.652931,360.800203
3,3,oDesk,web,Redwood City,USA,37.479952,-122.178557,0,23,24.078317,682.174891
4,4,Bazaarvoice,software,Austin,USA,30.407545,-97.717667,0,36,23.260582,1040.105325
...,...,...,...,...,...,...,...,...,...,...,...
80,80,Virdia,cleantech,Redwood City,USA,37.523311,-122.259154,0,23,15.519179,658.716597
81,81,Buzzient,enterprise,Boston,USA,42.362856,-71.083762,0,26,4.689089,811.573218
82,82,Nuventix,cleantech,Austin,USA,30.239229,-97.834211,0,36,16.757988,810.101295
83,83,Pearl Therapeutics,biotech,Redwood City,USA,37.502169,-122.217727,0,23,19.833559,NaN


In [10]:
print(offices.dtypes)

Unnamed: 0           int64
name                object
category            object
city                object
country             object
latitude           float64
longitude          float64
is_gaming            int64
total_companies      int64
dist_airport       float64
dist_starbucks     float64
dtype: object


In [11]:
##Function to find closest whatever place we want with nearbyserach gmaps query
def closest_gtype(office,query,radius,gtype):
    coords_off = (office.get('latitude'),office.get('longitude'))
    coords_off2 = ("{},{}".format(coords_off[0],coords_off[1]))
    nbs_list=fs.nearbysearchType(query,coords_off2,radius,gtype)
    dist = []
    a=0
    if nbs_list==[]:
        dist.append(a)
    else:
        df=pd.DataFrame(fs.getLocation(nbs_list)).iloc[0]
        coords_df=(df.get('latitude'),df.get('longitude'))
        geo_dist=(geopy.distance.geodesic(coords_off, coords_df).m)
        dist.append(geo_dist)
    return dist[0]

In [12]:
## 4. Distance to Closest Vegan Restaurant
dist_vegan=[]
for x in range(len(offices)):
    office=offices.iloc[x]
    dist_vegan.append(fs.closest_gtype(office,"vegan",500,"restaurant"))

offices['dist_vegan'] = dist_vegan

In [13]:
## 5. Distance to Closest Waldorf school
dist_school=[]
for x in range(len(offices)):
    office=offices.iloc[x]
    dist_school.append(fs.closest_gtype(office,"waldorf",4000,"school"))

offices['dist_school'] = dist_school

In [22]:
display(offices)

,Unnamed: 0,name,category,city,country,latitude,longitude,is_gaming,total_companies,dist_airport,dist_starbucks,dist_vegan,dist_school,rank
17,17,Qik,mobile,Redwood City,USA,37.486368,-122.230116,0,23,20.194579,15.727920,156.383293,635.725913,22
68,68,SCVNGR,games_video,Boston,USA,42.340653,-71.068003,1,26,4.764528,593.158218,40.895928,1892.669333,22
73,73,Shareaholic,web,Boston,USA,42.364756,-71.103259,0,26,6.238827,1225.057454,80.897534,2312.798032,20
35,35,Optaros,consulting,Boston,USA,42.356812,-71.058185,0,26,2.979413,147.519598,174.009406,2820.901034,20
2,2,Carbonite,network_hosting,Boston,USA,42.350571,-71.076869,0,26,4.652931,360.800203,177.114680,1132.796641,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,71,SeeWhy,analytics,Boston,USA,42.675142,-71.147213,0,26,35.099558,NaN,330.739653,3340.286081,4
33,33,Moblyng,games_video,Redwood City,USA,37.453665,-122.229069,1,23,22.961189,NaN,473.459014,3251.792952,4
15,15,Yodle,advertising,Austin,USA,30.378658,-97.731249,0,36,20.512108,NaN,401.192421,2553.419202,4
1,1,RockYou,games_video,Redwood City,USA,37.484619,-122.206893,1,23,21.824782,1392.763284,388.944091,3390.832648,4


In [23]:
##6. Ranking based on functions_rank
offices['rank'] = offices.apply (lambda row: fr.rank_airport(row), axis=1) + offices.apply (lambda row: fr.rank_starbucks(row), axis=1) + offices.apply (lambda row: fr.rank_vegan(row), axis=1) + offices.apply (lambda row: fr.rank_schools(row), axis=1)
offices=offices.sort_values('rank',ascending=False)
display(offices.head(20))
print(offices.dtypes)

,Unnamed: 0,name,category,city,country,latitude,longitude,is_gaming,total_companies,dist_airport,dist_starbucks,dist_vegan,dist_school,rank
17,17,Qik,mobile,Redwood City,USA,37.486368,-122.230116,0,23,20.194579,15.727920,156.383293,635.725913,22
68,68,SCVNGR,games_video,Boston,USA,42.340653,-71.068003,1,26,4.764528,593.158218,40.895928,1892.669333,22
73,73,Shareaholic,web,Boston,USA,42.364756,-71.103259,0,26,6.238827,1225.057454,80.897534,2312.798032,20
35,35,Optaros,consulting,Boston,USA,42.356812,-71.058185,0,26,2.979413,147.519598,174.009406,2820.901034,20
2,2,Carbonite,network_hosting,Boston,USA,42.350571,-71.076869,0,26,4.652931,360.800203,177.114680,1132.796641,19
72,72,VKernel Corporation,software,Boston,USA,42.349737,-71.049190,0,26,2.995129,379.312019,112.627774,3342.111334,17
67,67,Tatto Media,advertising,Boston,USA,42.353491,-71.056465,0,26,3.090434,105.232815,265.314992,2830.020365,17
40,40,Jumptap,mobile,Boston,USA,42.368633,-71.075305,0,26,3.903119,146.335896,331.349684,2713.999533,17
10,10,Eons,network_hosting,Boston,USA,42.375219,-71.056101,0,26,2.339181,640.936151,113.054956,1728.679696,17
62,62,WordStream,advertising,Boston,USA,42.354257,-71.056605,0,26,3.044372,143.974665,418.268080,2843.147698,17


Unnamed: 0           int64
name                object
category            object
city                object
country             object
latitude           float64
longitude          float64
is_gaming            int64
total_companies      int64
dist_airport       float64
dist_starbucks     float64
dist_vegan         float64
dist_school        float64
rank                 int64
dtype: object


In [27]:
##An the winner is: 
office1=offices.iloc[0]
print(type(office1))
print(offices.iloc[0])

<class 'pandas.core.series.Series'>
Unnamed: 0                   17
name                        Qik
category                 mobile
city               Redwood City
country                     USA
latitude                37.4864
longitude               -122.23
is_gaming                     0
total_companies              23
dist_airport            20.1946
dist_starbucks          15.7279
dist_vegan              156.383
dist_school             635.726
rank                         22
Name: 17, dtype: object


In [26]:
map_office=folium.Map(location=[37.4864,-122.23],zoom_start=30)


map_office